In [10]:
import json
from datetime import date, datetime, timedelta


In [11]:
with open('2014-2015.json') as data_file:
    data = json.load(data_file)

In [12]:
month_lookup = {'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12,'Jan':1,'Feb':1,'Mar':3,'Apr':4,'May':5}

In [19]:
def game_date(string_date):
    if string_date[3:6] in ['Aug','Sep','Oct','Nov','Dec']:
        return date(2014,month_lookup[string_date[3:6]],int(string_date[0:2]))
    else:
        return date(2015,month_lookup[string_date[3:6]],int(string_date[0:2]))

In [14]:
class Played_game(object):
    def __init__(self, game_json):
        self.opp = game_json[2][:3]
        self.loc = game_json[2][4] # "A" for away, "H" for home
        self.points = game_json[19]
        self.minutes = game_json[3]
        self.date = game_date(game_json[0])
        

In [15]:
class Player(object):
    def __init__(self, player_json):
        self.raw = player_json
        self.name = u"{first_name} {second_name}".format(**player_json)
        self.cost = player_json["now_cost"]
        self.position = player_json["type_name"]
        self.games= [Played_game(g) for g in player_json[u'fixture_history'][u'all'] ]
    def __repr__(self):
        return "# %s" % self.name.encode("ascii","ignore")

In [20]:
players = [Player(x) for x in data.itervalues()]

In [21]:
print data[u'344'][u'fixture_history'][u'all'][0]

[u'17 Aug 13:30', 1, u'SOU(H) 2-1', 90, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 16, 22, 0, 55, 2]


In [22]:
def player_form(player, game):
    points  =  0
    games_played = 0
    for g in player.games:
        if g.date >= game.date + timedelta(days=-30) and g.date < game.date and g.minutes>0:
            points = points + g.points
            games_played = games_played + 1
    if points == 0:
        return 0
    else:
        return float(points)/games_played

In [28]:
def opp_form(game):
    points  =  0
    games_played = 0
    for player in players:
        for g in player.games:
            if g.date >= game.date + timedelta(days=-30) and g.date < game.date and g.minutes>0 and g.opp == game.opp:
                points = points + g.points
                games_played = games_played + 1
    if points == 0:
        return 0
    else:
        return float(points)/games_played

In [30]:
for g in players[111].games:
    print g.date,g.loc, g.minutes, g.points,player_form(players[111],g),g.opp, team_form(g.opp,g)

2014-08-16 H 90 2 0 HUL 0
2014-08-24 A 90 2 2.0 TOT 1.85714285714
2014-08-30 H 0 0 2.0 SUN 2.5
2014-09-13 H 19 5 2.0 SWA 1.64285714286
2014-09-21 A 0 0 3.5 MCI 2.90476190476
2014-09-27 H 10 1 5.0 AVL 2.78571428571
2014-10-05 H 0 0 3.0 ARS 1.92857142857
2014-10-18 A 90 2 1.0 CRY 3.02380952381
2014-10-26 A 0 0 1.5 MUN 2.075
2014-11-01 H 0 0 2.0 QPR 2.975
2014-11-08 A 0 0 2.0 LIV 3.21428571429
2014-11-22 H 12 1 0 WBA 3.07142857143
2014-11-29 A 15 1 1.0 SUN 2.65853658537
2014-12-03 H 24 5 1.0 TOT 2.09756097561
2014-12-06 A 0 0 2.33333333333 NEW 2.64150943396
2014-12-13 H 0 0 2.33333333333 HUL 3.50909090909
2014-12-22 A 0 0 2.33333333333 STK 3.01449275362
2014-12-26 H 0 0 3.0 WHU 1.78260869565
2014-12-28 A 2 1 3.0 SOU 3.64102564103
2015-01-01 A 0 0 3.0 TOT 2.91463414634
2015-01-10 H 7 1 1.0 NEW 3.18072289157
2015-01-17 A 16 1 1.25 SWA 2.57142857143
2015-01-31 H 86 7 1.25 MCI 2.51219512195
2015-01-07 A 27 1 1.0 AVL 4.32098765432
2015-01-11 H 69 2 1.0 EVE 3.66315789474
2015-01-21 H 0 0 1.2 BU